## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**План действий для выполнения поставленной задачи:**
1. Открыть файл с данными и изучить общую информацию
2. Провести предобработку данных:
    - обработать пропуски;
    - обработать ошибки в данных;
    - обработать дубликаты;
    - заменить тип данных (при необходимости);
    - провести лемматизацию;
    - провести категоризацию данных.
3. Проанализировать данные и ответить на вопросы:
    - Есть ли зависимость между наличием детей и возвратом кредита в срок?
    - Есть ли зависимость между семейным положением и возвратом кредита в срок?
    - Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
    - Как разные цели кредита влияют на его возврат в срок?
4. Сделать выводы и дать рекомендации.

<a id="1"></a>
### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [ ]:
import pandas as pd
from IPython.display import display #импорт метода display из библиотеки IPython.display для лучшей визуализации таблиц
data = pd.read_csv('/datasets/data.csv')
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [ ]:
data.duplicated().sum()

54

### Вывод

1. В файле обнаружены данные 21525 клиентов. Информация о каждом клиенте представлена в 12-ти столбцах(4 из которых относятся к количественным признакам, 8 к категориальным).
2. В столбцах days_employed и total_income есть пропущенные значения. Далее предстоим обработать пропуски и понять случайны они или нет.
3. В столбце education данные записаны в разных регистрах (прим. Среднее, среднее, СРЕДНЕЕ). Это мешает обработке данных, особенно если необходимо будет подсчиать количество строк по средним образованием.
4. В столбце days_employed есть отрицательные значения (а трудовой стаж не может быть отрицательным)
5. Значения переменных children (количество детей), dob_years (возраст), education_id (идентификатор образования), family_status_id (идентификатор семейного положения), debt (имел ли задолженность по возврату кредитов) представлены типом данных - "int", что корректно.
6. Значения переменных days_employed (трудовой стаж), total_income (доход в месяц) представлены типом данных "float". Т.к. излишние цифры после запятой могут отвлекать, то в разделе "Замена типа данных" заменим тип данных на "int".
7. Значения переменных education (образование клиента), family_status (семейное положение), gender (пол клиента), income_type (тип занятости), purpose (цель получения кредита) представлены типом данных "object", что вполне корректно.
8. Также в данных были найдены дубликаты - после выполнения действий в п.2-7 необходимо их еще раз подсчитать и избавиться от них.

<a id="2"></a>
### Шаг 2. Предобработка данных

### Обработка пропусков

Пропуски в данных, которые были выявлены в столбцах days_employed и total_income являются количественными переменными.
Для начала посчитаем количество пропусков в таблице

In [ ]:
print(data.isnull().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


Проверим у каких клиентов есть пропущенные значения в обеих столбца (days_employed и total_income) 

In [ ]:
print(len(data['days_employed'][data.isnull()['total_income'] == True]))

2174


Количество пропусков в обеих столбцах равно количеству пропусков в одном столбце, значит пропуски есть у одних и тех же клиентов.

Посмотрим на 10 клиентов с пропущенными значениями, чтобы понять есть ли у них еще что-то общее

In [ ]:
data[data.isnull()['total_income'] == True].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Проверим зависимость количества пропусков от возраста.
Для этого разобьем возраст на возрастные группы и подсчитаем отношение ratio количества пропусков к общему числу значений в группе.

In [ ]:
data_copy = data.copy()
data_copy['age_group'] = pd.qcut(data_copy['dob_years'],5) 
data_nan_age = data_copy.loc[data_copy['total_income'].isnull(), 'age_group'].value_counts().reset_index()
data_age = data_copy['age_group'].value_counts().reset_index()
data_nan_age['ratio'] = data_nan_age['age_group'] / data_age['age_group']
data_nan_age.round(2)

,index,age_group,ratio
0,"(47.0, 56.0]",451,0.10
1,"(31.0, 39.0]",442,0.10
2,"(39.0, 47.0]",441,0.10
3,"(-0.001, 31.0]",432,0.10
4,"(56.0, 75.0]",408,0.11


Пропуски встречаются во всех возрастных группах и отношение во всех возрастных группах примерно одинаковое 0,10. Зависимости от возраста нет.
Посмотрим есть ли зависимость пропусков от пола.

In [ ]:
data_nan_gen = data_copy.loc[data_copy['total_income'].isnull(), 'gender'].value_counts().reset_index()
data_gen = data_copy['gender'].value_counts().reset_index()
data_nan_gen['ratio'] = data_nan_gen['gender'] / data_gen['gender']
data_nan_gen.round(2)

,index,gender,ratio
0,F,1484,0.10
1,M,690,0.09


Тут также видим, что отношение количества клиентов с пропусками к общему числу клиентов по каждому полу примерно равно. Значит, зависимости пропусков от пола нету.
Посмотрит есть ли зависимость пропусков от семейного положения.

In [ ]:
data_nan_fam = data_copy.loc[data_copy['total_income'].isnull(), 'family_status'].value_counts().reset_index()
data_fam = data_copy['family_status'].value_counts().reset_index()
data_nan_fam['ratio'] = data_nan_fam['family_status'] / data_fam['family_status']
data_nan_fam.round(2)

,index,family_status,ratio
0,женат / замужем,1237,0.10
1,гражданский брак,442,0.11
2,Не женат / не замужем,288,0.10
3,в разводе,112,0.09
4,вдовец / вдова,95,0.10


Тут также зависимости нет. 
Проверим есть ли зависимость от типа занятости.

In [ ]:
data_nan_type = data_copy.loc[data_copy['total_income'].isnull(), 'income_type'].value_counts().reset_index()
data_type = data_copy['income_type'].value_counts().reset_index()
data_nan_type['ratio'] = data_nan_type['income_type'] / data_type['income_type']
data_nan_type.round(2)

,index,income_type,ratio
0,сотрудник,1105,0.10
1,компаньон,508,0.10
2,пенсионер,413,0.11
3,госслужащий,147,0.10
4,предприниматель,1,0.50


Тут выделяется только предприниматели. Но т.к. в данных их всего два, и у одного есть пропущенные значения, то это не говорит о том, что предприниматели чаще пропускают значения/ либо есть какая-либо зависимость пропусков от типа занятости. У остальных типов занятости отношение пропусков к количеству около 0,1. Т.е. зависимости также нет.

Проверим также есть ли зависимость пропусков от уровня образования

In [ ]:
data_copy['education'] = data_copy['education'].str.lower() #приведем все значения образования к нижнему регистру
data_nan_edu = data_copy.loc[data_copy['total_income'].isnull(), 'education'].value_counts().reset_index()
data_edu= data_copy['education'].value_counts().reset_index()
data_nan_edu['ratio'] = data_nan_edu['education'] / data_edu['education']
data_nan_edu.round(2)

,index,education,ratio
0,среднее,1540,0.10
1,высшее,544,0.10
2,неоконченное высшее,69,0.09
3,начальное,21,0.07


Немного меньше пропусков у клиентов с начильным образованием, но в целом ярко выраженной зависимости также нет.

**Вывод:**
Пропуски в столбцах (доход за месяц и трудовой стаж) есть у одних и тех же клиентов.
Пропуски выявлены у клиентов разного возраста, образования, семейного положения и типа занятости. Доля пропусков в разных группах примерно одинаковая, т.е. скорее всего данные не заполнены по ошибке или случайно.
Причины пропусков могут быть следующие:
1. Одни и те же клиенты не заполнили эту информацию в анкете (не захотели указывать по каким-либо причинам)
2. Техническия ошибка выгрузки данных из каких-то конкретных отделений банка.
3. Случайная составляющая (человеческий фактор)
4. Возможно есть зависимость от других переменных, которые нет в данных, например сумма займа. Возможно клиенты не указывают эту информации при взятии небольших кредитов

В качестве метода обработка пропусков - попробуем восстановить даннные и зменим пропущенные значения медианами по каждому типу занятости. Разбиение по типам занятости делаем для повышения точности и корректности восставленных значений.

Но для начала перед обработкой пропусков - удалим из данных дубликаты.

In [ ]:
data = data.drop_duplicates().reset_index(drop=True) #удалим дубликаты
print(data.duplicated().sum()) #Проверим, что дубликатов больше нет

0


Посмотрим основные статические данные по количественным переменным, чтобы выявить возможные ошибки.

In [ ]:
data[['children', 'days_employed', 'dob_years', 'total_income']].describe()

,children,days_employed,dob_years,total_income
count,21471.000000,19351.000000,21471.000000,1.935100e+04
mean,0.539565,63046.497661,43.279074,1.674223e+05
std,1.382978,140827.311974,12.574291,1.029716e+05
min,-1.000000,-18388.949901,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,2.265604e+06


1. Столбец children (количество детей):
    - в столбце присутствуют отрицательные значения, чего не может быть.
    - максимальное значение количества детей равно 20, что также очень похоже на ошибку.
    - исправим данные ошибки на этапе "Замены типа данных"

2. Столбец days_employed (трудовой стаж в днях):
    - в столбце присутствуют отрицательные значения, чего не может быть. Отрицательные значения заменим в рамках этапа "Обработка пропусков".
    - максимальной значение равно 401755 дней или  1100 лет. Требуется уточнить у людей, кто предоставили данные, что подразумевается под этими числами. Возможно это количество дней от какой-то "базовой" даты. 

3. Столбец dob_years (возраст в годах):
    - найдено значение, равное 0. Возраст с таким значением быть не может. В дальнейшем на этапе "Обработка пропусков" исправим ошибку.

4. Столбец total_income (доход за месяц):
    - Разброс дохода достаточно большой от 20 тысяч до 2,2 млн. В дальнейшем при обработке пропуской это следует учесть и не использовать средние значения.

Избавимся от отрицательных значений в столбце трудовой стаж, возьмем значение по модулю. Затем проверим, посмотрев минимальное значение.

In [ ]:
data['days_employed'] = abs(data['days_employed']) 
#Проверим, посмотрев минимальное значение:
data['days_employed'].min()

24.14163324048118

Теперь приступим к обработке пропусков и найдем количество пропусков по каждому типу занятости

In [ ]:
data.loc[data['total_income'].isnull(), 'income_type'].value_counts()

сотрудник          1077
компаньон           503
пенсионер           394
госслужащий         145
предприниматель       1
Name: income_type, dtype: int64

У нас есть пропуски в следующих столбцах days_employed и total_income у следующих типов занятости: сотрудник, пенсионер, компаньон, госслужащий, предприниматель. Т.к. тип занятости влияет на уровень дохода, Заменим пропуски по каждому типу занятости соотвествующими медианами. Медиану выбираем, т.к. он среднее значения из-за разброса уровня дохода может дать менее релевантое значение.

In [ ]:
data_sotrudnik = data.loc[data['income_type'] == 'сотрудник']
data.loc[data['income_type'] == 'сотрудник'] = data_sotrudnik.fillna(data_sotrudnik.median())
data_pensioner = data.loc[data['income_type'] == 'пенсионер']
data.loc[data['income_type'] == 'пенсионер'] = data_pensioner.fillna(data_pensioner.median())
data_companyon = data.loc[data['income_type'] == 'компаньон']
data.loc[data['income_type'] == 'компаньон'] = data_companyon.fillna(data_companyon.median())
data_gossluga = data.loc[data['income_type'] == 'госслужащий']
data.loc[data['income_type'] == 'госслужащий'] = data_gossluga.fillna(data_gossluga.median())
data_businessman = data.loc[data['income_type'] == 'предприниматель']
data.loc[data['income_type'] == 'предприниматель'] = data_businessman.fillna(data_businessman.median())

print(data.isnull().sum())

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


In [ ]:
len(data_pensioner)

3837

Посмотрим количества повторения разных значений пола

In [ ]:
print(data['gender'].value_counts())

F      14189
M       7281
XNA        1
Name: gender, dtype: int64


Замечено станное значения XNA, т.к. оно всего одно, то не повлияет на результаты анализа. 
Удалим его из данных.

In [ ]:
data = data[data['gender'] != 'XNA']
data.index = pd.RangeIndex(0, len(data.index)) #сброс индексов
print(data['gender'].value_counts()) #проверка удаления

F    14189
M     7281
Name: gender, dtype: int64


Ранее было найдено некорректное значение возраста = 0. Посмотрим какой тип занятости имееют клиенты с таким возрастом и сколько их

In [ ]:
data.loc[data['dob_years'] == 0, 'income_type'].value_counts()

сотрудник      55
компаньон      20
пенсионер      20
госслужащий     6
Name: income_type, dtype: int64

Нашли 101 значение с возрастом равным нулю. Половина из них - это сотрудники компаний. Т.к. значение возраста не может равняться нулю, то предположим, что они не оставлены по ошибке или случайно. Заменим эти значения медианами по каждому типу занятости.

In [ ]:
#найдем медианы возраста по каждому типу занятости
dob_years_medians = data.groupby('income_type')['dob_years'].median()
print(dob_years_medians)

income_type
безработный        38.0
в декрете          39.0
госслужащий        40.0
компаньон          39.0
пенсионер          60.0
предприниматель    42.5
сотрудник          39.0
студент            22.0
Name: dob_years, dtype: float64


In [ ]:
#Заменим значения возраста "0" на полученные медианы
data.loc[(data['dob_years'] == 0) & (data['income_type'] == 'сотрудник'), 'dob_years'] = dob_years_medians[6]
data.loc[(data['dob_years'] == 0) & (data['income_type'] == 'пенсионер'), 'dob_years'] = dob_years_medians[4]
data.loc[(data['dob_years'] == 0) & (data['income_type'] == 'компаньон'), 'dob_years'] = dob_years_medians[3]
data.loc[(data['dob_years'] == 0) & (data['income_type'] == 'госслужащий'), 'dob_years'] = dob_years_medians[2]

In [ ]:
# Проверим, все ли нули мы заменили.
print('Минимальный возраст', data['dob_years'].min()) 

Минимальный возраст 19.0


Тип данных столбца возраста изменился на float. В будущем вернем ему обратно тип данных int.

### Вывод

В столбцах days_employed (трудовой стаж в днях) и total_income (доход за месяц) были найдены пропуски.
Т.к. мы анализируем данные клиентов банка, которые взяли кредит, то скорее всего они все же работаю и какую-то сумму в месяц зарабатывают, поэтому нулевые значения в этих столбцах будут не совсем корректными. 
Пэтому, чтобы не терять данные в остальных столбцах (строки с нулевыми значениями составляют примерно 10% от всех значений в таблице), мы посчитали медианы доходов за месяц по каждому типу занятости и заменили ими пропуски.
Также в столбце gender был найдено одно значение XNA, которое удалили из данных.
В столбце с возрастом были найдены значения, равным 0. Нашли медианы возрастов по каждому типу занятости и заменили ими нулевые.

### Замена типа данных

1. Избавимся от лишних цифр в доходе за месяц, и заменим тип данных на целочисленный.
2. Заменим тип данных возраста на целочисленный. В столбце количество детей в семье присутствуют два типа странных значений "-1" и "20". Чтобы избавиться от отрицательных значений возьмем их по модулю. Значение "20" явно ошибочное. Скорее всего опечатка при заполнении и тут либо 0, либо 2.Т.к. значений с "0" гораздо больше, и добавление 20-ти значения не сильно повлияет, допустим, что там хотели поставить 0.


In [ ]:
data['total_income'] = data['total_income'].astype('int') #Избавимся от лишних цифр в доходе за месяц
data['dob_years'] = data['dob_years'].astype('int') # заменим тип данных возраста на целочисленный
data['children'] = abs(data['children']) #избавимся от отрицательных значений
data['children'] = data['children'].replace(20, 0) #Заменим значение 20 на 0
print(data['children'].value_counts()) #Проверим корректность проведенных операций со столбцом children

0    14182
1     4856
2     2052
3      330
4       41
5        9
Name: children, dtype: int64


Используем метод try-except и подсчитаем количество корректных значений в столбце days_employed после замены типа

In [ ]:
count_lines = 0
total_position = 0
wrong_lines = 0
for row in data['days_employed']: 
    try:
        count_lines += 1
        level = int(row) #в этой переменной сохраните позицию в выдаче 
        total_position += level  #сложите все позиции в этой переменной 
        #result = total_position / count_lines
#print(result)
    except: 
        wrong_lines += 1
    
print('Суммарное количество значений {}'.format(count_lines))
print('Количество некорректных значений {}'.format(wrong_lines))

Суммарное количество значений 21470
Количество некорректных значений 0


Все значения в столбце days_employed корректы. Заменим тип данных float на int при помощи astype().

In [ ]:
data['days_employed'] = data['days_employed'].astype('int') #
print(data['days_employed'].head()) #проверим, выведим первые 10 значений

0      8437
1      4024
2      5623
3      4124
4    340266
Name: days_employed, dtype: int64


In [ ]:
data.dtypes #проверка изменения типов данных

children             int64
days_employed        int64
dob_years            int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
debt                 int64
total_income         int64
purpose             object
dtype: object

### Вывод

1. В столбце трудовой стаж в днях было большое количество знаков после запятой. Т.к. отрицательного трудового стажа не бывает были взяты модули этих значений. Далее заметили тип данных трудового стажа на целочисленный int.
2. В столбце ежемесячного дохода был заменен тип данных на целочисленный, т.к. было большое количество знаков после запятой и такая точность была не нужна.
3. Тип данных столбца возраста был заменен на целочисленный.

### Обработка дубликатов

In [ ]:
print(data['education'].unique())

['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']


В столбце education есть названия, которые отличаются только регистром, чтобы учесть такие дубликаты
при дальнейшем анализе - создадим столбец education_lowercase, в котором переведем названия столбца education в нижний регистр. Подсчитаем количество клиентов с каждым типом образования.

In [ ]:
data['education_lowercase'] = data['education'].str.lower()
print(data['education_lowercase'].value_counts())

среднее                15188
высшее                  5251
неоконченное высшее      743
начальное                282
ученая степень             6
Name: education_lowercase, dtype: int64


Посчитаем количество дубликатов

In [ ]:
data.duplicated().sum() 

0

### Вывод

Ранее в данных были выявлены дубликаты (54 шт.), которые были удалены при помощи drop_duplicates() перед обработкой пропусков. После замены нулевых значений, замены типов данных и изменения регистра в столбце образования - новых дублкатов не появилось.
Одной из причин ранее обнаруженных дубликатов может быть техническая ошибка при выгрузке.

### Лемматизация

In [ ]:
# импортируем библиотеку pymystem3 для лемматизации:
from pymystem3 import Mystem
m = Mystem()
from collections import Counter

Посмотрим уникальные значения целей займа 

In [ ]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [ ]:
#lemmas = m.lemmatize(data['purpose'][2])
#Напишем функицию для лемматизации конкретной строки
def lemmatize_func(i):
    lemmas = m.lemmatize(data['purpose'][i]) # i - номер строки в столбце цель кредита
    print(lemmas) #получаем леммы
    print(Counter(lemmas)) #подсчитаем количество лемм
#Проверим функцию на строке 300
lemmas_test = lemmatize_func(300)  
print(lemmas_test)
                     

['на', ' ', 'проведение', ' ', 'свадьба', '\n']
Counter({' ': 2, 'на': 1, 'проведение': 1, 'свадьба': 1, '\n': 1})
None


In [ ]:
lemmas = []
for i in range(len(data)):
    lemmas += m.lemmatize(data['purpose'][i])
print(Counter(lemmas))
    

Counter({' ': 33595, '\n': 21470, 'недвижимость': 6352, 'покупка': 5899, 'жилье': 4461, 'автомобиль': 4308, 'образование': 4014, 'с': 2918, 'операция': 2604, 'свадьба': 2335, 'свой': 2231, 'на': 2228, 'строительство': 1879, 'высокий': 1374, 'получение': 1315, 'коммерческий': 1312, 'для': 1290, 'жилой': 1231, 'сделка': 941, 'дополнительный': 907, 'заниматься': 904, 'проведение': 773, 'сыграть': 769, 'сдача': 652, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'подержанный': 486, 'подержать': 478, 'приобретение': 461, 'профильный': 436})


Выделим популярные цели отдельно

In [ ]:
categories = ['недвижимость', 'жилье', 'автомобиль', 'образование', 'операция', 'свадьба', 'строительство']

Проведем лемматизацию, дновременно заменив полученный список лемм в каждой строке на главное ключевое слово из списка категорий

In [ ]:
def lemmatize_change(text):
    lemma = m.lemmatize(text)
    for word in categories:
        if word in lemma:
            lemma = word
    return lemma

data['purpose_group'] = data['purpose'].apply(lemmatize_change)        
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_lowercase,purpose_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,высшее,жилье
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,среднее,автомобиль
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,среднее,жилье
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,среднее,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,среднее,свадьба


Посчитаем количества возможных категорий целей займа.

In [ ]:
data['purpose_group'].value_counts()

недвижимость    6352
жилье           4461
автомобиль      4308
образование     4014
свадьба         2335
Name: purpose_group, dtype: int64

Т.к. жилье и недвижимость почти одно и тоже, то произведем замену

In [ ]:
data.loc[data['purpose_group'] == 'жилье', 'purpose_group'] = 'недвижимость'
data['purpose_group'].value_counts()

недвижимость    10813
автомобиль       4308
образование      4014
свадьба          2335
Name: purpose_group, dtype: int64

### Вывод

Провели лемматизацию столбца purpose (цель получения кредита) и выявили, что популярными целями являются: недвижимость, автомобиль, образование, свадьба.

### Категоризация данных

Категоризация по типу.
Создадим словари и оптимизируем данные
Создадим словарь соответствия семейного статусу и id семейного статуса

In [ ]:
data_family_dict = data[['family_status', 'family_status_id']].drop_duplicates().reset_index(drop=True)
display(data_family_dict)

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


Создадим словарь соответствия уровня образования и id образования 

In [ ]:
data_edu_dict = data[['education_lowercase', 'education_id']].drop_duplicates().reset_index(drop=True)
display(data_edu_dict)

,education_lowercase,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [ ]:
data_log = data[['children', 'dob_years', 'education_id', 'family_status_id', 'debt', 'total_income', 'purpose']]
data_log.head(10)

,children,dob_years,education_id,family_status_id,debt,total_income,purpose
0,1,42,0,0,0,253875,покупка жилья
1,1,36,1,0,0,112080,приобретение автомобиля
2,0,33,1,0,0,145885,покупка жилья
3,3,32,1,0,0,267628,дополнительное образование
4,0,53,1,1,0,158616,сыграть свадьбу
5,0,27,0,1,0,255763,покупка жилья
6,0,43,0,0,0,240525,операции с жильем
7,0,50,1,0,0,135823,образование
8,2,35,0,1,0,95856,на проведение свадьбы
9,0,41,1,0,0,144425,покупка жилья для семьи


In [ ]:
data_log['debt'].value_counts()

0    19729
1     1741
Name: debt, dtype: int64

Напишем функция, которая оценивает уровень дохода:
- если доход за месяц меньше 75% клиентов, то возвращает 'низкий доход' (т.е. получится 25% клиентов с самым низкий доходом)
- если доход за месяц больше клиентов, то возвращает 'высокий доход' (25% клиентов с самым высоким доходом)
- если доход по середине (между квантилями 0.25 и 0.75), то возвращает 'средний доход'

In [ ]:
pd.options.mode.chained_assignment = None  # default='warn' - отключим предупреждение SettingWithCopyWarning
data_log_quan_25 = data_log['total_income'].quantile(0.25) #квантиль 25% 
data_log_quan_75 = data_log['total_income'].quantile(0.75) #квантиль 75%
def income(total):
    if total <= data_log_quan_25:
        return 'низкий'
    if total > data_log_quan_75:
        return 'высокий'
    return 'средний'
data_log['type_total_income'] = data_log['total_income'].apply(income)
data_log.head()

,children,dob_years,education_id,family_status_id,debt,total_income,purpose,type_total_income
0,1,42,0,0,0,253875,покупка жилья,высокий
1,1,36,1,0,0,112080,приобретение автомобиля,средний
2,0,33,1,0,0,145885,покупка жилья,средний
3,3,32,1,0,0,267628,дополнительное образование,высокий
4,0,53,1,1,0,158616,сыграть свадьбу,средний


### Вывод

В рамках категоризации:
1. Был создан словарь соответствия семейного статусу и id семейного статуса 
2. Был создан словарь соответствия уровня образования и id образования
3. Данные были оптимизированы. В таблицу data_log не вносились текстовые столбцы семейного статуса и уровня образования с целью оптимизации используемой памяти.

<a id="3"></a>
### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Построим сводную таблицу количество детей и количество задолженностей и найдем отношение ratio кол-ва клиентов с задолженностью к общему числу клиентов и добавим в таблицу

In [ ]:
clients_value_child = data['children'].value_counts() # #количество клиентов по количеству детей
#Построим сводную таблицу количество детей и количество задолженностей
data_pivot1 = data.pivot_table(index='children', values='debt', aggfunc='sum')
#найдем отношение ratio кол-ва клиентов с задолженностью к общему числу клиентов и добавим в таблицу
data_pivot1['ratio'] = (data_pivot1['debt'] / clients_value_child).round(3)
data_pivot1.sort_values(by='ratio', ascending=False) #выведем результат на экран и отсортируем его по убыванию столбца ratio

,debt,ratio
children,,
4,4,0.098
2,194,0.095
1,445,0.092
3,27,0.082
0,1071,0.076
5,0,0.000


### Вывод

Клиенты с детьми чаще имеют задолженность по займу, чем те, у кого их нет. (исключение составили только клиенты с 5-ю детьми)

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Построим сводную таблицу семейного положения и количество задолженностей и найдем отношение ratio кол-ва клиентов с задолженностью к общему числу клиентов и добавим в таблицу.

In [ ]:
clients_value_family = data['family_status'].value_counts() # #количество клиентов по семейному положению
#Построим сводную таблицу семейного положения и количество задолженностей
data_pivot2 = data.pivot_table(index = 'family_status', values = 'debt', aggfunc = 'sum')
#найдем отношение ratio кол-ва клиентов с задолженностью к общему числу клиентов и добавим в таблицу
data_pivot2['ratio'] = (data_pivot2['debt'] / clients_value_family).round(3)
data_pivot2.sort_values(by='ratio', ascending=False) #выведем результат на экран и отсортируем его по убыванию столбца ratio

,debt,ratio
family_status,,
Не женат / не замужем,274,0.098
гражданский брак,388,0.093
женат / замужем,931,0.075
в разводе,85,0.071
вдовец / вдова,63,0.066


### Вывод

Клиенты без офицального брака (не женат/не замужем или даже гражданский брак) чаще имеют задолженность по займам, чем те, кто состоит или состоял в официальном браке.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Построим сводную таблицу уровня дохода и количество задолженностей и найдем отношение ratio кол-ва клиентов с задолженностью к общему числу клиентов и добавим в таблицу.

In [ ]:
clients_value_income = data_log['type_total_income'].value_counts() # #количество клиентов по уровню дохода
#Построим сводную таблицу уровня дохода и количество задолженностей
data_pivot3 = data_log.pivot_table(index = 'type_total_income', values = 'debt', aggfunc = 'sum')
#найдем отношение ratio кол-ва клиентов с задолженностью к общему числу клиентов и добавим в таблицу
data_pivot3['ratio'] = (data_pivot3['debt'] / clients_value_income).round(3)
data_pivot3.sort_values(by='ratio', ascending=False) #выведем результат на экран и отсортируем его по убыванию столбца ratio

,debt,ratio
type_total_income,,
средний,931,0.087
низкий,427,0.080
высокий,383,0.071


### Вывод

По данным видно, что люди с высоким и низким достатком лучше возвращают займы, но это может быть связано с тем, что людям с высоким достатком возвращать займ проще, а люди с низкий достатком не берут больших займов. 
Чаще всего имеют задолженность представители со средним доходом. 

- Как разные цели кредита влияют на его возврат в срок?

Построим сводную таблицу цели займа и количество задолженностей и найдем отношение ratio кол-ва клиентов с задолженностью к общему числу клиентов и добавим в таблицу.

In [ ]:
clients_value_purpose = data['purpose_group'].value_counts() # #количество клиентов по типу цели займа
#Построим сводную таблицу цели займа и количество задолженностей
data_pivot4 = data.pivot_table(index = ['purpose_group'], values = 'debt', aggfunc = 'sum')
#найдем отношение ratio кол-ва клиентов с задолженностью к общему числу клиентов и добавим в таблицу
data_pivot4['ratio'] = (data_pivot4['debt'] / clients_value_purpose).round(3)
data_pivot4.sort_values(by='ratio', ascending=False)

,debt,ratio
purpose_group,,
автомобиль,403,0.094
образование,370,0.092
свадьба,186,0.080
недвижимость,782,0.072


### Вывод

Реже всего задолженность имеют клиенты, берущие займ на покупку недвижимости.
Чаще всего задолженность имеют клиенты, берущие займ на покупку автомобиля и на образование.

<a id="4"></a>
### Шаг 4. Общий вывод

В рамках задачи исследования надёжности заёмщиков:
1. Прочитали файл с данными клиентов и изучили общуюу информацию:
    - В файле нашли данные о 21525 клиентов банка. Информация о каждом клиенте представлена в 12-ти столбцах.
    - В данных были найдены пропущенные значения, в одном столбце данные были записаны в разных регистрах, также были найдены отрицательные значения и дубликаты.
2. Обработали пропущенные значения:
    - В столбцах days_employed (трудовой стаж в днях), total_income (доход за месяц), dob_years (возраст) были найдены пропуски и нулевые значения. Т.к. их характер скорее всего случайный, нулевые значения были заменены на медианы значений по каждому типу занятости.
    - Также в столбце gender был найдено одно значение XNA, которое удалили из данных.
3. Заменили типы данных в столбцах трудового стажа, возраста и ежемесячного дохода на целочисленный тип. 
4. Из данных были удалены дубликаты при помощи drop_duplicates().
Одной из причин дубликатов может быть техническая ошибка при выгрузке.
5. Провели лемматизацию столбца purpose (цель получения кредита) и выявили, что популярными целями являются: недвижимость, автомобиль, образование, свадьба.
6. Ответили на вопросы о зависимости возврата займа и определили, что на это влияет:
    - наличие детей
    - семеное положение
    - цели.
И наоборот не особо влияет:
    - уровень дохода.
7. В качестве рекомендаций:
    - чаще выдавать займы клиентам без детей, но состоявших хоть раз в официальном браке.
    - обращать внимание на доход, но не ркуводствоваться им как главным критерием для выдачи займа, т.к. клиенты с низким доходом возвращают займы почти также хорошо как и клиенты с высоким доходом.
    - чаще выдавать займ клиентам, которые берут его для попупки недвижимости.

